# Analysing Flight-delays data

# Table of contents<a class="anchor" id="table"></a>

* [1 Working with RDD](#1)
* [1.1 Data Preparation and Loading](#1.1)
* [1.1.1 Creating SparkSession & SparkContext](#OneOneOne)
* [1.1.2 Read CSV files, Preprocessing, and final(formatted data) RDD for each file](#OneOneTwo)
* [1.1.2.1 Flights RDD](#1.1.2.1)
* [1.1.2.2 Airports RDD](#1.1.2.2)
* [1.1.3 Show RDD number of columns, and number of records](#1.1.3)
* [1.2 Dataset flights partitioning](#1.2)
* [1.2.1 Obtain the maximum arrival time ](#1.2.1)
* [1.2.2 Obtain the maximum minimum time ](#1.2.2)
* [1.2.3 Define hash partitioning](#1.2.3)
* [1.2.4 Display the records in each partition](#1.2.4)
* [1.3 Query RDD](#1.3)
* [1.3.1 Collect a total number of flights for each month for all flights](#1.3.1)
* [1.3.2 Collect the average delay for each month for all flights](#1.3.2)
* [2 Working with DataFrames](#2)
* [2.1 Data Preparation and Loading](#2.1)
* [2.1.1 Define DataFrames](#2.1.1)
* [2.1.2 Display the Scheme of DataFrames](#2.1.2)
* [2.1.3 Transform date-time and location column](#2.1.3)
* [2.2.1 January Flights Events with ANC airport](#2.2.1)
* [2.2.2 Average Arrival Delay From Origin to Destination](#2.2.2)
* [2.2.3 Join Query with Airports DataFrame](#2.2.3)
* [2.3 Analysis](#2.3.1)
* [2.3.1 Relationship between day of week with mean arrival delay, total time delay, and count flights](#2.3.1)
* [2.3.2 Display mean arrival delay each month](#2.3.2)
* [2.3.3 Relationship between mean departure delay and mean arrival delay](#2.3.3)
* [3 RDDs vs DataFrame vs Spark SQL](#3)
* [3.1 RDD Operation](#3.1)
* [3.2 DataFrame Operation](#3.1)
* [3.3 Spark SQL Operation](#3.1)
* [3.4 Discussion](#3.1)


# 1 Working with RDD<a class="anchor" id="1"></a>
## 1.1 Data Preparation and Loading<a class="anchor" id="1.1"></a>
### 1.1.1 Create SparkSession and SparkContext<a class="anchor" id="OneOneOne"></a>
[Back to top](#table)

In [1]:
# Import SparkConf class into program
from pyspark import SparkConf

# local[*]: run Spark in local mode with as many working processors as logical cores on your machine
# If we want Spark to run locally with 'k' worker threads, we can specify as "local[k]".
master = "local[*]"
# The `appName` field is a name to be shown on the Spark cluster UI page
app_name = "Data Analysis"
# Setup configuration parameters for Spark
spark_conf = SparkConf().setMaster(master).setAppName(app_name)

# Import SparkContext and SparkSession classes
from pyspark import SparkContext # Spark
from pyspark.sql import SparkSession # Spark SQL

# Method 1: Using SparkSession
spark = SparkSession.builder.config(conf=spark_conf).getOrCreate()
sc = spark.sparkContext
sc.setLogLevel('ERROR')

### 1.1.2 Import CSV files and Make RDD for each file<a class="anchor" id="OneOneTwo"></a>
[Back to top](#table)

In [2]:
from pyspark.rdd import RDD

# This method is used to make rdd objects, load datasets, filter without headers and split into columns  
def makeRDD(filepath):
    # Importing the .csv files in the rdd object
    rdd = sc.textFile(filepath)
    
    # Taking the header row
    header = rdd.first()
    # Filtering the header row from the rest of the data
    rdd = rdd.filter(lambda x: x != header)

    # Splitting each line with comma to convert into rows and column 
    rdd = rdd.map(lambda line: line.split(','))
    
    # return the filterred rdd
    return rdd
    

### For Data Cleaning and Pre-processing the dataset

In [3]:
# Converting datatypes to int or float
def typeCastColumns(rdd):
    rdd = rdd.map(lambda x: (int(x[0]), int(x[1]), int(x[2]), int(x[3]), x[4], int(x[5]), x[6], x[7], x[8], x[9], x[10], 
                                        float(x[11].strip("")), float(x[12].strip("")), x[13], x[14], float(x[15].strip("")), float(x[16].strip("")), float(x[17].strip("")), x[18], float(x[19].strip("")), x[20],
                                         x[21], float(x[22].strip("")), x[23],x[24], x[25], x[26], x[27], x[28], x[29], x[30]))

    return rdd

#### 1.1.2.1 Flights RDD <a class="anchor" id="1.1.2.1"></a>
[Back to top](#table)

In [4]:
# Loading 20 .csv files into a single rdd object 
flights_rdd = makeRDD("Datasets/flight*.csv")

# flights_rdd = typeCastColumns(flights_rdd)

# This shows the first row of flights rdd
flights_rdd.take(1)

[['2015',
  '2',
  '6',
  '5',
  'OO',
  '6271',
  'N937SW',
  'FAR',
  'DEN',
  '1712',
  '1701',
  '-11',
  '15',
  '1716',
  '123',
  '117',
  '95',
  '627',
  '1751',
  '7',
  '1815',
  '1758',
  '-17',
  '0',
  '0',
  '',
  '',
  '',
  '',
  '',
  '']]

#### 1.1.2.2 Airports RDD <a class="anchor" id="1.1.2.2"></a>
[Back to top](#table)

In [5]:
# Loading the airsports .csv file into a rdd object
airports_rdd = makeRDD("Datasets/airports.csv")

# This shows the first 2 row of airports rdd
airports_rdd.take(2)

[['ABE',
  'Lehigh Valley International Airport',
  'Allentown',
  'PA',
  'USA',
  '40.65236',
  '-75.44040'],
 ['ABI',
  'Abilene Regional Airport',
  'Abilene',
  'TX',
  'USA',
  '32.41132',
  '-99.68190']]

### 1.1.3 Show RDD number of columns, and number of records <a class="anchor" id="1.1.3"></a>
[Back to top](#table)

In [6]:
def printDescription(rdd):
    print(f"Number of Records: {rdd.count()}")
    print(f"Number of Columns: {len(rdd.take(1)[0])}")
    print(f"Total partitions: {rdd.getNumPartitions()}")


In [7]:
print("Description for flights_rdd:")
printDescription(flights_rdd)

print("\nDescription for airports_rdd:")
printDescription(airports_rdd)

Description for flights_rdd:
Number of Records: 582184
Number of Columns: 31
Total partitions: 20

Description for airports_rdd:
Number of Records: 322
Number of Columns: 7
Total partitions: 2


### Observation:

In RDD, the data is partitioned by default using "Random Equal Partitioning", it is used when no partitioning technique is specified.

## 1.2 Dataset Partitioning <a class="anchor" id="1.2"></a>
### 1.2.1 Obtain the maximum arrival time <a class="anchor" id="1.2.1"></a>
[Back to top](#table)

In [8]:
%%time
# Removing null or empty values from the arrival delay column
max_arrival_delay = flights_rdd.filter(lambda x : x[22] not in  ["",''])

# Fetching the maximum arrival delay 
max_arrival_delay = max_arrival_delay.max(key=lambda x: float((x[22].strip(''))))

print("Maximum arrival delay using flights_rdd is", max_arrival_delay[22])

Maximum arrival delay using flights_rdd is 1665
CPU times: user 5.33 ms, sys: 3.31 ms, total: 8.64 ms
Wall time: 588 ms


### 1.2.2 Obtain the minimum arrival time <a class="anchor" id="1.2.2"></a>
[Back to top](#table)

In [9]:
%%time
# Removing null or empty values from the arrival delay column
min_arrival_delay = flights_rdd.filter(lambda x : x[22] not in  ["",''])

# Fetching the Minimum arrival delay 
min_arrival_delay = min_arrival_delay.min(key=lambda x: float((x[22].strip(''))))

print("Minimum arrival delay using flights_rdd is", min_arrival_delay[22])

Minimum arrival delay using flights_rdd is -82
CPU times: user 5.08 ms, sys: 2.92 ms, total: 8 ms
Wall time: 565 ms


### 1.2.3 Define hash partitioning function <a class="anchor" id="1.2.3"></a>
[Back to top](#table)

In [10]:
# Defining the hash function which takes the year as partitioning id and sum all the digits  
number_of_partitions = 4
def hash_function(year):
    print("year")
    total = 0
    for digit in str(year):
        total += int(digit)
    return (total%number_of_partitions)


hash_rdd = flights_rdd.partitionBy(number_of_partitions, hash_function)

In [11]:
def check_partition_records(rdd):
    partitions = rdd.glom().collect()
    for index,partition in enumerate(partitions):
        print(f'------ Partition {index}:')
        if len(partition) > 0:
            print(f"Partition {index}: {len(partition)} records")
            
    return partitions

In [12]:
%%time
# For Flights_rdd
print("Partitions for Flights_rdd: \n")
flights_partitions = check_partition_records(flights_rdd)

Partitions for Flights_rdd: 

------ Partition 0:
Partition 0: 28940 records
------ Partition 1:
Partition 1: 27545 records
------ Partition 2:
Partition 2: 29843 records
------ Partition 3:
Partition 3: 27039 records
------ Partition 4:
Partition 4: 30863 records
------ Partition 5:
Partition 5: 31056 records
------ Partition 6:
Partition 6: 30535 records
------ Partition 7:
Partition 7: 29844 records
------ Partition 8:
Partition 8: 27234 records
------ Partition 9:
Partition 9: 30542 records
------ Partition 10:
Partition 10: 30102 records
------ Partition 11:
Partition 11: 28532 records
------ Partition 12:
Partition 12: 28471 records
------ Partition 13:
Partition 13: 28081 records
------ Partition 14:
Partition 14: 31221 records
------ Partition 15:
Partition 15: 28472 records
------ Partition 16:
Partition 16: 30803 records
------ Partition 17:
Partition 17: 28615 records
------ Partition 18:
Partition 18: 29452 records
------ Partition 19:
Partition 19: 24994 records
CPU times:

In [13]:
%%time
# For Airports_rdd
print("Partitions for Airports_rdd: \n")
airports_partitions = check_partition_records(airports_rdd)

Partitions for Airports_rdd: 

------ Partition 0:
Partition 0: 162 records
------ Partition 1:
Partition 1: 160 records
CPU times: user 7.16 ms, sys: 3.74 ms, total: 10.9 ms
Wall time: 44.2 ms


## Observation:

The data present in each hash partitioning contains uneven number of distribution of rocords suggesting that it is skewed. For instance in flights_rdd, some partitions like Partition 17 contains 24994 records whereas Partition 0 contains 31221 records. To overcome this skewness we can alter the partitioning with the use of range partitioning or round robin equal partitioning.

For airports_rdd, the records seems almost equally divided with 162 to 160 records in Partition 0 and 1 respectively.

### 1.2.4 Display the records in each partition <a class="anchor" id="1.2.4"></a>
[Back to top](#table)

In [14]:
def displayPartitionRecords(rdd, partitions):
    for index,partition in enumerate(partitions):
        print(f'------ Partition {index}:')
        for record in partition:
            print(record)

In [15]:
# %%time
# For Flights_rdd
# print("Partitions for Flights_rdd: \n")
# displayPartitionRecords(flights_rdd, flights_partitions)

In [16]:
%%time
# For airports_rdd
print("Partitions for airports_rdd: \n")
displayPartitionRecords(airports_rdd, airports_partitions)

Partitions for airports_rdd: 

------ Partition 0:
['ABE', 'Lehigh Valley International Airport', 'Allentown', 'PA', 'USA', '40.65236', '-75.44040']
['ABI', 'Abilene Regional Airport', 'Abilene', 'TX', 'USA', '32.41132', '-99.68190']
['ABQ', 'Albuquerque International Sunport', 'Albuquerque', 'NM', 'USA', '35.04022', '-106.60919']
['ABR', 'Aberdeen Regional Airport', 'Aberdeen', 'SD', 'USA', '45.44906', '-98.42183']
['ABY', 'Southwest Georgia Regional Airport', 'Albany', 'GA', 'USA', '31.53552', '-84.19447']
['ACK', 'Nantucket Memorial Airport', 'Nantucket', 'MA', 'USA', '41.25305', '-70.06018']
['ACT', 'Waco Regional Airport', 'Waco', 'TX', 'USA', '31.61129', '-97.23052']
['ACV', 'Arcata Airport', 'Arcata/Eureka', 'CA', 'USA', '40.97812', '-124.10862']
['ACY', 'Atlantic City International Airport', 'Atlantic City', 'NJ', 'USA', '39.45758', '-74.57717']
['ADK', 'Adak Airport', 'Adak', 'AK', 'USA', '51.87796', '-176.64603']
['ADQ', 'Kodiak Airport', 'Kodiak', 'AK', 'USA', '57.74997', '-

## Observation:

The **Hash partitioning in RDD** uses the formula <code>partition = hash_function() % numPartitions</code> in order to determine which record falls into which partition. All the records within the partition has the same hash value. For larger datasets, its difficult to view the records in the partition. Moreover,  there might be unequal distribution of 
records for the datasets which brings skewness and effective utilization of processors.

Therefore, we may use **Range partitioning in RDD** which distributes the records in the partition based on their range of key.  

## 1.3 Query RDD  <a class="anchor" id="1.3"></a>
### 1.3.1 Collect a total number of flights for each month <a class="anchor" id="1.3.1"></a>
[Back to top](#table)

In [17]:
# This method is used to return total number of flights per month
# month: integer value of month
def number_of_flights_per_month(month):
    # Filtering the rdd(the MONTH column based on the given month) 
    rdd = flights_rdd.filter(lambda x: int(x[1]) == month)
    
    # returning the number of records(i.e. the total number of flights)
    return rdd.count()

In [18]:
# Creating a list of months
month_list = ["January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]

# iterating over the month list to fetch the total number of flights
for month in range(0, len(month_list)):
    number_of_flights = number_of_flights_per_month(month + 1)
    print("The total number of flights in the month of", month_list[month], " are ", number_of_flights)


The total number of flights in the month of January  are  47136
The total number of flights in the month of February  are  42798
The total number of flights in the month of March  are  50816
The total number of flights in the month of April  are  48810
The total number of flights in the month of May  are  49691
The total number of flights in the month of June  are  50256
The total number of flights in the month of July  are  52065
The total number of flights in the month of August  are  50524
The total number of flights in the month of September  are  46733
The total number of flights in the month of October  are  48680
The total number of flights in the month of November  are  46809
The total number of flights in the month of December  are  47866


## Observation:

It can be analysed there are maximum number of flights in the month of **July** with **52065 flights** and **February** being the minimum with **42798 flights**.

### 1.3.2 Collect the average delay for each month <a class="anchor" id="1.3.2"></a>
[Back to top](#table)

In [19]:
# This method is used to return average delay of flights per month
# month: integer value of month
def average_delay_per_month(month):
    # Filtering the rdd(the MONTH column based on the given month) 
    rdd = flights_rdd.filter(lambda x: int(x[1]) == month)
    
    # The map function is used to extract and removing the empty values from the column
    rdd = rdd.map(lambda x : float(x[23].strip("")))
    
    # returning the number of records(i.e. the total number of flights)
    return rdd.mean()


In [20]:
# iterating over the month list to fetch the total number of flights
for month in range(0, len(month_list)):
    average_delay = average_delay_per_month(month + 1)
    print("The average delay of flights in the month of", month_list[month], "are", average_delay)


The average delay of flights in the month of January are 0.0017608621860149376
The average delay of flights in the month of February are 0.002313192205243238
The average delay of flights in the month of March are 0.0021449937027707834
The average delay of flights in the month of April are 0.002704363859864785
The average delay of flights in the month of May are 0.0032802720814634487
The average delay of flights in the month of June are 0.003541865647882843
The average delay of flights in the month of July are 0.002957841160088355
The average delay of flights in the month of August are 0.0032063969598606643
The average delay of flights in the month of September are 0.001497870883529842
The average delay of flights in the month of October are 0.0017050123253903064
The average delay of flights in the month of November are 0.0019654339977354805
The average delay of flights in the month of December are 0.0033217732837504733


## Observation:

It can be analysed there are maximum delay in flights are received in the month of **July** with **0.0035418 units** and **October** being the **minimum with 0.001705 units**

# 2 Working with DataFrame <a class="anchor" id="2"></a>
## 2.1. Data Preparation and Loading <a class="anchor" id="2.1"></a>
### 2.1.1 Define dataframes and loading scheme<a class="anchor" id="2.1.1"></a>
[Back to top](#table)

In [21]:
# loading units data into one data frame
flightsDf = spark.read.csv("Datasets/flight*.csv",header=True, inferSchema=True)

# loading crash data into one data frame
airportsDf = spark.read.csv("Datasets/airports.csv",header=True, inferSchema=True)

### 2.1.2 Display the schema of the final two dataframes<a class="anchor" id="2.1.2"></a>
[Back to top](#table)

In [22]:
flightsDf.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [23]:
# .explain() method is used to verify the partitioning and the query plan when an action is performed
flightsDf.explain()

== Physical Plan ==
FileScan csv [YEAR#16,MONTH#17,DAY#18,DAY_OF_WEEK#19,AIRLINE#20,FLIGHT_NUMBER#21,TAIL_NUMBER#22,ORIGIN_AIRPORT#23,DESTINATION_AIRPORT#24,SCHEDULED_DEPARTURE#25,DEPARTURE_TIME#26,DEPARTURE_DELAY#27,TAXI_OUT#28,WHEELS_OFF#29,SCHEDULED_TIME#30,ELAPSED_TIME#31,AIR_TIME#32,DISTANCE#33,WHEELS_ON#34,TAXI_IN#35,SCHEDULED_ARRIVAL#36,ARRIVAL_TIME#37,ARRIVAL_DELAY#38,DIVERTED#39,... 7 more fields] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/navalaggarwal/Desktop/WEBSITE projects/Flight Delays in USA/Data An..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<YEAR:int,MONTH:int,DAY:int,DAY_OF_WEEK:int,AIRLINE:string,FLIGHT_NUMBER:int,TAIL_NUMBER:st...




In [24]:
airportsDf.printSchema()

root
 |-- IATA_CODE: string (nullable = true)
 |-- AIRPORT: string (nullable = true)
 |-- CITY: string (nullable = true)
 |-- STATE: string (nullable = true)
 |-- COUNTRY: string (nullable = true)
 |-- LATITUDE: double (nullable = true)
 |-- LONGITUDE: double (nullable = true)



In [25]:
# .explain() method is used to verify the partitioning and the query plan when an action is performed
airportsDf.explain()

== Physical Plan ==
FileScan csv [IATA_CODE#94,AIRPORT#95,CITY#96,STATE#97,COUNTRY#98,LATITUDE#99,LONGITUDE#100] Batched: false, DataFilters: [], Format: CSV, Location: InMemoryFileIndex[file:/Users/navalaggarwal/Desktop/WEBSITE projects/Flight Delays in USA/Data An..., PartitionFilters: [], PushedFilters: [], ReadSchema: struct<IATA_CODE:string,AIRPORT:string,CITY:string,STATE:string,COUNTRY:string,LATITUDE:double,LO...




## 2.2. Query Analysis <a class="anchor" id="2.2"></a>
### 2.2.1 January flight events with ANC airport <a class="anchor" id="2.2.1"></a>
[Back to top](#table)

In [26]:
# This is used to view how the dataframe looks before making any query
flightsDf.toPandas().head()

,YEAR,MONTH,DAY,DAY_OF_WEEK,AIRLINE,FLIGHT_NUMBER,TAIL_NUMBER,ORIGIN_AIRPORT,DESTINATION_AIRPORT,SCHEDULED_DEPARTURE,...,ARRIVAL_TIME,ARRIVAL_DELAY,DIVERTED,CANCELLED,CANCELLATION_REASON,AIR_SYSTEM_DELAY,SECURITY_DELAY,AIRLINE_DELAY,LATE_AIRCRAFT_DELAY,WEATHER_DELAY
0,2015,6,26,5,EV,4951,N707EV,BHM,LGA,630,...,950.0,-15.0,0,0,None,NaN,NaN,NaN,NaN,NaN
1,2015,12,19,6,WN,3589,N764SW,MKE,DCA,1630,...,1903.0,-22.0,0,0,None,NaN,NaN,NaN,NaN,NaN
2,2015,1,10,6,WN,3370,N720WN,SNA,OAK,1055,...,1214.0,-6.0,0,0,None,NaN,NaN,NaN,NaN,NaN
3,2015,11,1,7,WN,2081,N461WN,PDX,LAS,1345,...,1541.0,-4.0,0,0,None,NaN,NaN,NaN,NaN,NaN
4,2015,6,11,4,WN,836,N8628A,BNA,BOS,1635,...,1956.0,-9.0,0,0,None,NaN,NaN,NaN,NaN,NaN


In [27]:
# For accessing the column of the dataframe
from pyspark.sql.functions import col

# Filtering the dataset based on year, month and origin_airport
janFlightEventsAncDf = flightsDf.filter((col("YEAR") == 2015) & (col("MONTH") == 1) & (col("ORIGIN_AIRPORT") == "ANC"))

# Selecting the specified columns
janFlightEventsAncDf = janFlightEventsAncDf.select(["MONTH", "ORIGIN_AIRPORT", "DESTINATION_AIRPORT", "DISTANCE","ARRIVAL_DELAY"])

# Checking the number of records
print(janFlightEventsAncDf.count())
# Displaying the result
janFlightEventsAncDf.show()

115
+-----+--------------+-------------------+--------+-------------+
|MONTH|ORIGIN_AIRPORT|DESTINATION_AIRPORT|DISTANCE|ARRIVAL_DELAY|
+-----+--------------+-------------------+--------+-------------+
|    1|           ANC|                SEA|    1448|          -13|
|    1|           ANC|                SEA|    1448|           -4|
|    1|           ANC|                JNU|     571|           17|
|    1|           ANC|                CDV|     160|           20|
|    1|           ANC|                BET|     399|          -20|
|    1|           ANC|                SEA|    1448|          -15|
|    1|           ANC|                SEA|    1448|          -11|
|    1|           ANC|                ADQ|     253|          -16|
|    1|           ANC|                SEA|    1448|           17|
|    1|           ANC|                BET|     399|           -9|
|    1|           ANC|                SEA|    1448|           15|
|    1|           ANC|                FAI|     261|           -6|
|    1

## Observation:

It shows the number of flight taking place from **ANC** airport events in January 2015

### 2.2.2 Average Arrival Delay From Origin to Destination <a class="anchor" id="2.2.2"></a>
[Back to top](#table)

In [28]:
import pyspark.sql.functions as F

# Grouping the dataset and applying average on the arrival_delay and setting this column as average_delay 
# and ordering it based on average_delay 
janFlightEventsAncAvgDf = janFlightEventsAncDf.groupby(["ORIGIN_AIRPORT", "DESTINATION_AIRPORT"]) \
                            .agg(F.mean("ARRIVAL_DELAY").alias('AVERAGE_DELAY')) \
                            .orderBy("AVERAGE_DELAY")

# Checking the number of records
print(janFlightEventsAncAvgDf.count())
# Displaying the result
janFlightEventsAncAvgDf.show()


18
+--------------+-------------------+-------------------+
|ORIGIN_AIRPORT|DESTINATION_AIRPORT|      AVERAGE_DELAY|
+--------------+-------------------+-------------------+
|           ANC|                ADK|              -27.0|
|           ANC|                HNL|              -20.0|
|           ANC|                MSP|             -19.25|
|           ANC|                BET| -9.090909090909092|
|           ANC|                SEA| -6.490196078431373|
|           ANC|                BRW| -4.333333333333333|
|           ANC|                OME|               -3.0|
|           ANC|                ADQ|-2.6666666666666665|
|           ANC|                CDV|                1.0|
|           ANC|                OTZ|               1.25|
|           ANC|                PHX|                2.0|
|           ANC|                DEN| 3.3333333333333335|
|           ANC|                PDX|                3.5|
|           ANC|                JNU|                5.0|
|           ANC|            

## Observation:

It shows the average delay of flights taking place from **ANC** airport events in January 2015 with **ADK being at -27 units** and to **FAI at 25 units**

### 2.2.3 Join Query with Airports DataFrame <a class="anchor" id="2.2.3"></a>
[Back to top](#table)

In [29]:
# This is used to view how the dataframe looks before making any query
airportsDf.toPandas().head()

,IATA_CODE,AIRPORT,CITY,STATE,COUNTRY,LATITUDE,LONGITUDE
0,ABE,Lehigh Valley International Airport,Allentown,PA,USA,40.65236,-75.44040
1,ABI,Abilene Regional Airport,Abilene,TX,USA,32.41132,-99.68190
2,ABQ,Albuquerque International Sunport,Albuquerque,NM,USA,35.04022,-106.60919
3,ABR,Aberdeen Regional Airport,Aberdeen,SD,USA,45.44906,-98.42183
4,ABY,Southwest Georgia Regional Airport,Albany,GA,USA,31.53552,-84.19447


In [30]:
# Joining the airports dataframe and janFlightEventsAncAvgDf using inner join with key as IATA_CODE and the Destination_Airport 
joinedSqlDf =  airportsDf.join(janFlightEventsAncAvgDf,airportsDf.IATA_CODE==janFlightEventsAncAvgDf.DESTINATION_AIRPORT,how='inner')

# Checking the number of records
print(joinedSqlDf.count())

# Displaying the result
joinedSqlDf.show()

18
+---------+--------------------+-------------+-----+-------+--------+----------+--------------+-------------------+-------------------+
|IATA_CODE|             AIRPORT|         CITY|STATE|COUNTRY|LATITUDE| LONGITUDE|ORIGIN_AIRPORT|DESTINATION_AIRPORT|      AVERAGE_DELAY|
+---------+--------------------+-------------+-----+-------+--------+----------+--------------+-------------------+-------------------+
|      BRW|Wiley Post-Will R...|       Barrow|   AK|    USA|71.28545|  -156.766|           ANC|                BRW| -4.333333333333333|
|      ADK|        Adak Airport|         Adak|   AK|    USA|51.87796|-176.64603|           ANC|                ADK|              -27.0|
|      OME|        Nome Airport|         Nome|   AK|    USA| 64.5122|-165.44525|           ANC|                OME|               -3.0|
|      JNU|Juneau Internatio...|       Juneau|   AK|    USA|58.35496|-134.57628|           ANC|                JNU|                5.0|
|      LAS|McCarran Internat...|    Las Vegas

In [31]:
# .explain() method is used to verify the partitioning and the query plan when an action is performed
joinedSqlDf.explain()

== Physical Plan ==
*(3) BroadcastHashJoin [IATA_CODE#94], [DESTINATION_AIRPORT#24], Inner, BuildLeft, false
:- BroadcastExchange HashedRelationBroadcastMode(List(input[0, string, false]),false), [id=#322]
:  +- *(1) Filter isnotnull(IATA_CODE#94)
:     +- FileScan csv [IATA_CODE#94,AIRPORT#95,CITY#96,STATE#97,COUNTRY#98,LATITUDE#99,LONGITUDE#100] Batched: false, DataFilters: [isnotnull(IATA_CODE#94)], Format: CSV, Location: InMemoryFileIndex[file:/Users/navalaggarwal/Desktop/WEBSITE projects/Flight Delays in USA/Data An..., PartitionFilters: [], PushedFilters: [IsNotNull(IATA_CODE)], ReadSchema: struct<IATA_CODE:string,AIRPORT:string,CITY:string,STATE:string,COUNTRY:string,LATITUDE:double,LO...
+- *(3) HashAggregate(keys=[ORIGIN_AIRPORT#23, DESTINATION_AIRPORT#24], functions=[avg(cast(ARRIVAL_DELAY#38 as bigint))])
   +- Exchange hashpartitioning(ORIGIN_AIRPORT#23, DESTINATION_AIRPORT#24, 200), ENSURE_REQUIREMENTS, [id=#329]
      +- *(2) HashAggregate(keys=[ORIGIN_AIRPORT#23, DESTINA

## Observation:

As the number of records in janFlightEventsAncAvgDf is 18 (i.e. smaller dataset), therefore Spark uses Broadcast Hash join to join these dataframes. 

## 2.3. Analysis <a class="anchor" id="2.3"></a>
### 2.3.1 Relationship between day of week with mean arrival delay, total time delay, and count flights <a class="anchor" id="2.3.1"></a>
[Back to top](#table)

In [32]:
# Converting the dataframes into SQL formats
flightsDf.createOrReplaceTempView("sql_flights")
airportsDf.createOrReplaceTempView("sql_airports")

In [33]:
# Creating SQL query 
sql_flights_per_day = spark.sql('''
  SELECT DAY_OF_WEEK, mean(ARRIVAL_DELAY) as MeanArrivalDelay, count(ARRIVAL_DELAY) as TotalTimeDelay,
  count(FLIGHT_NUMBER) as NumOfFlights
  
  FROM sql_flights
  WHERE YEAR == 2015
  GROUP BY DAY_OF_WEEK
  ORDER BY NumOfFlights DESC

''')
# Displaying the number of rows
print(sql_flights_per_day.count())
sql_flights_per_day.show()


7
+-----------+------------------+--------------+------------+
|DAY_OF_WEEK|  MeanArrivalDelay|TotalTimeDelay|NumOfFlights|
+-----------+------------------+--------------+------------+
|          4| 5.684831897201573|         86227|       87683|
|          1| 5.883000999381335|         84052|       86317|
|          5| 4.715112525093624|         85181|       86253|
|          3|3.9745505431431147|         84324|       85607|
|          2| 4.391518272706391|         82719|       84449|
|          7| 4.299206488272548|         79898|       81422|
|          6| 1.813841449342257|         69328|       70453|
+-----------+------------------+--------------+------------+



## Observation:

It shows that the number of flights is maximum on **Thursday (87683)** followed by **Monday (86317)**  and **Friday (86354)** and the minimum flights on **Saturday (70453)**. This could be explained by the fact that people plan their flights on Thursday and get the returning flights on Monday. 

It shows that the the total time delay is maximum on **Thursday** followed by **Friday**  and **Wednesday** and the minimum flights on **Saturday** and **Sunday**. It appears that on weekdays there is a longer time delay as compared to weekends.

It shows that the the mean arrival delay(arrival time - schedule arrival) is maximum on **Monday** followed by **Thursday**  and **Friday** and the minimum flights on **Saturday**.

### 2.3.2 Display mean arrival delay each month <a class="anchor" id="2.3.2"></a>
[Back to top](#table)

In [34]:
# Creating SQL query 
sql_flights_per_months = spark.sql('''
  SELECT MONTH, mean(ARRIVAL_DELAY) as MeanArrivalDelay, count(ARRIVAL_DELAY) as TotalTimeDelay,
  count(FLIGHT_NUMBER) as NumOfFlights
  
  FROM sql_flights
  GROUP BY MONTH
  ORDER BY MeanArrivalDelay

''')
# Displaying the number of rows
print(sql_flights_per_months.count())
sql_flights_per_months.show()


12
+-----+-------------------+--------------+------------+
|MONTH|   MeanArrivalDelay|TotalTimeDelay|NumOfFlights|
+-----+-------------------+--------------+------------+
|    9|-0.8498676252179341|         46459|       46733|
|   10| -0.541989784312509|         48357|       48680|
|   11| 0.8313745860658399|         46203|       46809|
|    4|  3.173803944339603|         48221|       48810|
|    5| 4.7121097658084405|         48977|       49691|
|    8|  4.713893233866763|         49866|       50524|
|    3|  5.011173860427592|         49580|       50816|
|    1|  5.804357298474946|         45900|       47136|
|   12|   6.15837046195826|         46909|       47866|
|    7|  6.786093552465234|         51415|       52065|
|    2|  8.123906203913085|         40684|       42798|
|    6|  9.747630090727856|         49158|       50256|
+-----+-------------------+--------------+------------+



## Observation:

It shows that the average arrival delay is minimum on **September** followed by **October**  and **November** and the maximum arrival delay on **June**.  

It shows that the total time delay is minimum on **February** followed by **January**  and **November** and the maximum arrival delay on **July** and **August**. 

It shows that the number of flights is minimum on **February** followed by **September**  and **November** and the maximum arrival delay on **July**. 

### 2.3.3 Relationship between mean departure delay and mean arrival delay <a class="anchor" id="2.3.3"></a>
[Back to top](#table)

In [35]:
# Creating SQL query 
sql_flights_per_months_delays = spark.sql('''
  SELECT MONTH, mean(ARRIVAL_DELAY) as MeanArrivalDelay, mean(DEPARTURE_DELAY) as MeanDeptDelay
  
  FROM sql_flights
  GROUP BY MONTH
  ORDER BY MeanDeptDelay DESC

''')
# Displaying the number of rows
print(sql_flights_per_months_delays.count())
sql_flights_per_months_delays.show()

12
+-----+-------------------+------------------+
|MONTH|   MeanArrivalDelay|     MeanDeptDelay|
+-----+-------------------+------------------+
|    6|  9.747630090727856|  13.9730063585922|
|   12|   6.15837046195826|11.821651454043728|
|    7|  6.786093552465234|11.708608758020432|
|    2|  8.123906203913085|11.620796080832823|
|    8|  4.713893233866763|10.086906141367324|
|    1|  5.804357298474946|  9.75401499511029|
|    3|  5.011173860427592| 9.718308159530178|
|    5| 4.7121097658084405| 9.550310180006102|
|    4|  3.173803944339603| 7.737554783759199|
|   11| 0.8313745860658399| 6.630585898709037|
|   10| -0.541989784312509| 5.243436261558784|
|    9|-0.8498676252179341| 4.728506981740065|
+-----+-------------------+------------------+



In [36]:
# Creating SQL query 
sql_flights_per_months_delays = spark.sql('''
  SELECT MONTH, mean(ARRIVAL_DELAY) as MeanArrivalDelay, mean(DEPARTURE_DELAY) as MeanDeptDelay
  
  FROM sql_flights
  GROUP BY MONTH
  ORDER BY MeanArrivalDelay DESC

''')
# Displaying the number of rows
print(sql_flights_per_months_delays.count())
sql_flights_per_months_delays.show()

12
+-----+-------------------+------------------+
|MONTH|   MeanArrivalDelay|     MeanDeptDelay|
+-----+-------------------+------------------+
|    6|  9.747630090727856|  13.9730063585922|
|    2|  8.123906203913085|11.620796080832823|
|    7|  6.786093552465234|11.708608758020432|
|   12|   6.15837046195826|11.821651454043728|
|    1|  5.804357298474946|  9.75401499511029|
|    3|  5.011173860427592| 9.718308159530178|
|    8|  4.713893233866763|10.086906141367324|
|    5| 4.7121097658084405| 9.550310180006102|
|    4|  3.173803944339603| 7.737554783759199|
|   11| 0.8313745860658399| 6.630585898709037|
|   10| -0.541989784312509| 5.243436261558784|
|    9|-0.8498676252179341| 4.728506981740065|
+-----+-------------------+------------------+



## Observation:

It shows that the average dept delay is maximum on **June** followed by **December**  and **July** and the minimum dept delay on **September**.  


It shows that the average arrival delay is maximum on **June** followed by **February**  and **July** and the minimum arrival delay on **September**.  

This explains that both average dept delay and average arrival delay have maximum in June and minimum in September. There could be a linear relation between both both months.

# 3 RDDs vs DataFrame vs Spark SQL <a class="anchor" id="3"></a>


Implement the following queries using RDDs, DataFrames and SparkSQL separately. Log the time taken for each query in each approach using the “%%time” built-in magic command in Jupyter Notebook and discuss the performance difference of these 3 approaches.

<strong>Find the MONTH and DAY_OF_WEEK, number of flights, and average delay where TAIL_NUMBER = ‘N407AS’. Note number of flights and average delay should be aggregated separately. The average delay should be grouped by both MONTH and DAYS_OF_WEEK.</strong>

## 3.1 RDD Operation<a class="anchor" id="3.1"></a>
[Back to top](#table)

In [37]:
flightsDf.printSchema()

root
 |-- YEAR: integer (nullable = true)
 |-- MONTH: integer (nullable = true)
 |-- DAY: integer (nullable = true)
 |-- DAY_OF_WEEK: integer (nullable = true)
 |-- AIRLINE: string (nullable = true)
 |-- FLIGHT_NUMBER: integer (nullable = true)
 |-- TAIL_NUMBER: string (nullable = true)
 |-- ORIGIN_AIRPORT: string (nullable = true)
 |-- DESTINATION_AIRPORT: string (nullable = true)
 |-- SCHEDULED_DEPARTURE: integer (nullable = true)
 |-- DEPARTURE_TIME: integer (nullable = true)
 |-- DEPARTURE_DELAY: integer (nullable = true)
 |-- TAXI_OUT: integer (nullable = true)
 |-- WHEELS_OFF: integer (nullable = true)
 |-- SCHEDULED_TIME: integer (nullable = true)
 |-- ELAPSED_TIME: integer (nullable = true)
 |-- AIR_TIME: integer (nullable = true)
 |-- DISTANCE: integer (nullable = true)
 |-- WHEELS_ON: integer (nullable = true)
 |-- TAXI_IN: integer (nullable = true)
 |-- SCHEDULED_ARRIVAL: integer (nullable = true)
 |-- ARRIVAL_TIME: integer (nullable = true)
 |-- ARRIVAL_DELAY: integer (null

In [38]:
%%time
tail_number_rdd = flights_rdd.filter(lambda x: x[6] == "N407AS")


CPU times: user 13 µs, sys: 1 µs, total: 14 µs
Wall time: 15 µs


## 3.2 DataFrame Operation<a class="anchor" id="3.2"></a>
[Back to top](#table)

In [39]:
%%time

# Filtering the dataset based on year, month and origin_airport
tailNumberDf = flightsDf.filter(col("TAIL_NUMBER") == "N407AS")

tailNumberDf = tailNumberDf.groupby(["MONTH", "DAY_OF_WEEK","TAIL_NUMBER"]) \
                            .agg(F.mean("ARRIVAL_DELAY").alias('MeanArrivalDelay'), 
                                F.mean("DEPARTURE_DELAY").alias('MeanDeptDelay'), 
                                F.count("FLIGHT_NUMBER").alias('NumOfFlights'))

# Selecting the specified columns
tailNumberDf = tailNumberDf.select(["MONTH","DAY_OF_WEEK", "NumOfFlights", "MeanArrivalDelay", "MeanDeptDelay"])

print(tailNumberDf.count())
# Displaying the result
tailNumberDf.show()


66
+-----+-----------+------------+-------------------+------------------+
|MONTH|DAY_OF_WEEK|NumOfFlights|   MeanArrivalDelay|     MeanDeptDelay|
+-----+-----------+------------+-------------------+------------------+
|   12|          4|           1|                6.0|               2.0|
|    4|          6|           1|              -20.0|               1.0|
|   12|          7|           2|               -1.0|              -2.0|
|    1|          1|           1|               -6.0|               4.0|
|    3|          4|           1|                2.0|               1.0|
|    1|          6|           3|  4.333333333333333| 8.666666666666666|
|    3|          2|           2|              -28.0|              -5.5|
|    9|          4|           3|-10.666666666666666|-8.333333333333334|
|    5|          6|           2|               -3.0|               0.5|
|   11|          7|           3|               -4.0|              -5.0|
|   12|          2|           2|              -11.5|         

## 3.3 Spark SQL OPERATION<a class="anchor" id="3.3"></a>
[Back to top](#table)

In [40]:
%%time
# Creating SQL query 
sql_tail_number = spark.sql('''
  SELECT MONTH, DAY_OF_WEEK,
  mean(ARRIVAL_DELAY) as MeanArrivalDelay, mean(DEPARTURE_DELAY) as MeanDeptDelay, 
  count(FLIGHT_NUMBER) as NumOfFlights
  
  FROM sql_flights
  WHERE TAIL_NUMBER == "N407AS"
  GROUP BY MONTH, DAY_OF_WEEK, TAIL_NUMBER

''')
# Displaying the number of rows
print(sql_tail_number.count())
sql_tail_number.show()

66
+-----+-----------+-------------------+------------------+------------+
|MONTH|DAY_OF_WEEK|   MeanArrivalDelay|     MeanDeptDelay|NumOfFlights|
+-----+-----------+-------------------+------------------+------------+
|   12|          4|                6.0|               2.0|           1|
|    4|          6|              -20.0|               1.0|           1|
|   12|          7|               -1.0|              -2.0|           2|
|    1|          1|               -6.0|               4.0|           1|
|    3|          4|                2.0|               1.0|           1|
|    1|          6|  4.333333333333333| 8.666666666666666|           3|
|    3|          2|              -28.0|              -5.5|           2|
|    9|          4|-10.666666666666666|-8.333333333333334|           3|
|    5|          6|               -3.0|               0.5|           2|
|   11|          7|               -4.0|              -5.0|           3|
|   12|          2|              -11.5|               1.0|   

## 3.4 Discussion<a class="anchor" id="3.4"></a>
[Back to top](#table)

For certain queries RDD outperforms the dataframe and SQL queries but it is very difficult to construct programatically and requeries in depth of RDD structures.

In terms of performance, there is no much significant difference between queries through Dataframe or SQL as the execution engine and internal data structures are same for both the methods.

The data frame queries are much easier to construct programmatically whereas SQL queries are more concise and easier to comprehend.